In [15]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Binning.pickle')

data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
3,Masters,2016,A,3,27,Male,No,5,1,"(2015.0, 2016.0]","(26.0, 27.0]","(4.0, 5.0]"
5,Bachelors,2016,A,3,22,Male,No,0,0,"(2015.0, 2016.0]","(21.999, 24.7]","(-0.001, 1.0]"
6,Bachelors,2015,B,3,38,Male,No,0,0,"(2014.0, 2015.0]","(34.0, 38.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4647,Bachelors,2016,C,3,30,Male,No,2,0,"(2015.0, 2016.0]","(28.0, 31.0]","(1.0, 2.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(24.7, 26.0]","(3.0, 4.0]"
4649,Masters,2013,C,2,37,Male,No,2,1,"(2011.999, 2013.0]","(34.0, 38.0]","(1.0, 2.0]"
4650,Masters,2018,B,3,27,Male,No,5,1,"(2017.0, 2018.0]","(26.0, 27.0]","(4.0, 5.0]"


In [16]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w
dt_train = pd.DataFrame()
dt_train['LeaveOrNot'] = data['LeaveOrNot']
ATTRS = ['Education', 'PaymentTier','Age','Gender','EverBenched']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train


WOE Education -> {'Bachelors': -0.15886505087872912, 'PHD': -0.4709699310516477, 'Masters': 0.6615439092921434}
WOE PaymentTier -> {1: 0.04773347096871903, 2: 1.004973626480855, 3: -0.30473556270485164}
WOE Age -> {22: 0.23076099319968923, 23: -0.12691345107212668, 24: 0.24968900308520808, 25: 0.3757867652499468, 26: -0.008763779297979113, 27: 0.13805988865422922, 28: -0.1931889295825783, 29: -0.23639768391381516, 30: 0.014916744367090925, 31: -0.23227396672995299, 32: -0.014796152951420689, 33: -0.3012668382169046, 34: -0.06237492993455561, 35: -0.04930284836720284, 36: -0.09412362824913592, 37: -0.08876168510775033, 38: -0.2752253539255523, 39: -0.12341082052092464, 40: -0.0008316126010482027, 41: -0.37252985823839513}
WOE Gender -> {'Female': 0.5076309567927609, 'Male': -0.3859132160750363}
WOE EverBenched -> {'No': -0.06996982236309017, 'Yes': 0.5458079572574848}


,LeaveOrNot,Education_WOE,PaymentTier_WOE,Age_WOE,Gender_WOE,EverBenched_WOE
1,1,-0.158865,0.047733,-0.193189,0.507631,-0.069970
2,0,-0.158865,-0.304736,-0.275225,0.507631,-0.069970
3,1,0.661544,-0.304736,0.138060,-0.385913,-0.069970
5,0,-0.158865,-0.304736,0.230761,-0.385913,-0.069970
6,0,-0.158865,-0.304736,-0.275225,-0.385913,-0.069970
...,...,...,...,...,...,...
4647,0,-0.158865,-0.304736,0.014917,-0.385913,-0.069970
4648,0,-0.158865,-0.304736,-0.008764,0.507631,-0.069970
4649,1,0.661544,1.004974,-0.088762,-0.385913,-0.069970
4650,1,0.661544,-0.304736,0.138060,-0.385913,-0.069970


In [17]:
Y =  dt_train['LeaveOrNot'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['PaymentTier_WOE'].to_numpy()
x_3 = dt_train['Age_WOE'].to_numpy()
x_4 = dt_train['Gender_WOE'].to_numpy()
x_5 = dt_train['EverBenched_WOE'].to_numpy()

array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5])
X = np.transpose(X)
X

array([[ 1.        , -0.15886505,  0.04773347, -0.19318893,  0.50763096,
        -0.06996982],
       [ 1.        , -0.15886505, -0.30473556, -0.27522535,  0.50763096,
        -0.06996982],
       [ 1.        ,  0.66154391, -0.30473556,  0.13805989, -0.38591322,
        -0.06996982],
       ...,
       [ 1.        ,  0.66154391,  1.00497363, -0.08876169, -0.38591322,
        -0.06996982],
       [ 1.        ,  0.66154391, -0.30473556,  0.13805989, -0.38591322,
        -0.06996982],
       [ 1.        , -0.15886505, -0.30473556,  0.01491674, -0.38591322,
         0.54580796]])

In [18]:
np.matmul(np.transpose(X), X)


array([[ 3.25800000e+03, -4.60692612e+01, -9.40900754e+01,
        -1.95071635e+01, -9.39107449e+01, -1.49025695e+01],
       [-4.60692612e+01,  3.61307507e+02,  1.49615389e+02,
         4.75195010e+00,  7.69032584e+00,  8.18382190e-01],
       [-9.40900754e+01,  1.49615389e+02,  8.75198187e+02,
         5.40150014e+00,  1.98046634e+02,  2.60727870e+00],
       [-1.95071635e+01,  4.75195010e+00,  5.40150014e+00,
         1.26448249e+02,  2.60732950e+00,  3.87028673e-01],
       [-9.39107449e+01,  7.69032584e+00,  1.98046634e+02,
         2.60732950e+00,  6.26816467e+02, -1.37108158e+00],
       [-1.49025695e+01,  8.18382190e-01,  2.60727870e+00,
         3.87028673e-01, -1.37108158e+00,  1.17332089e+02]])

In [19]:
np.linalg.inv(np.matmul(np.transpose(X), X))


array([[ 3.09503013e-04,  3.02046698e-05,  1.86654512e-05,
         4.48703874e-05,  4.00009139e-05,  3.90045350e-05],
       [ 3.02046698e-05,  2.99307476e-03, -5.39217303e-04,
        -8.76336962e-05,  1.38530447e-04, -3.15010840e-06],
       [ 1.86654512e-05, -5.39217303e-04,  1.32997278e-03,
        -2.51132890e-05, -4.10758339e-04, -2.81391420e-05],
       [ 4.48703874e-05, -8.76336962e-05, -2.51132890e-05,
         7.92007718e-03, -1.72547862e-05, -1.94582355e-05],
       [ 4.00009139e-05,  1.38530447e-04, -4.10758339e-04,
        -1.72547862e-05,  1.72958349e-03,  3.35098882e-05],
       [ 3.90045350e-05, -3.15010840e-06, -2.81391420e-05,
        -1.94582355e-05,  3.35098882e-05,  8.52887474e-03]])

In [20]:
np.matmul(np.transpose(X), Y)


array([1109.        ,   69.0943947 ,  173.98487989,   21.8670871 ,
        109.04229127,   22.77524508])

In [21]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.35480464, 0.15960365, 0.16885819, 0.21020128, 0.17145058,
       0.2356183 ])

In [22]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x
    return y

eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.355+ (0.160)x_1+ (0.169)x_2+ (0.210)x_3+ (0.171)x_4+ (0.236)x_5


In [23]:
data_test = pd.read_pickle('Data_test.pickle')

attr = pd.read_pickle('attr.pickle')
attr

,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,AGE_BIN,"(21.999, 24.7]",127,199,0.114518,0.092601,0.212426
1,AGE_BIN,"(24.7, 26.0]",285,476,0.256988,0.221498,0.148615
2,AGE_BIN,"(26.0, 27.0]",157,265,0.141569,0.123313,0.138060
3,AGE_BIN,"(27.0, 28.0]",134,315,0.120830,0.146580,-0.193189
4,AGE_BIN,"(28.0, 31.0]",126,279,0.113616,0.129828,-0.133386
5,AGE_BIN,"(31.0, 34.0]",83,180,0.074842,0.083760,-0.112572
6,AGE_BIN,"(34.0, 38.0]",122,269,0.110009,0.125174,-0.129146
7,AGE_BIN,"(38.0, 41.0]",75,166,0.067628,0.077245,-0.132956
8,EXPERIENCEINCURRENTDOMAIN_BIN,"(-0.001, 1.0]",208,426,0.187556,0.198232,-0.055357
9,EXPERIENCEINCURRENTDOMAIN_BIN,"(1.0, 2.0]",277,488,0.249775,0.227082,0.095246


In [24]:
# y_predict
data_test['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['PaymentTier_WOE'] \
         + B[3]*dt_train['Age_WOE'] + B[4]*dt_train['Gender_WOE'] + B[5]*dt_train['EverBenched_WOE']


data_test['P'] = 1/(1+np.exp(-data_test['y_predict']))

data_test['predict'] = data_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
data_test

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,y_predict,P,predict
0,Bachelors,2017,A,3,34,Male,No,0,0,NaN,NaN,0
4,Masters,2017,C,3,24,Male,Yes,2,1,NaN,NaN,0
11,Bachelors,2016,C,3,34,Male,No,3,0,NaN,NaN,0
14,Bachelors,2012,A,3,37,Male,No,4,0,NaN,NaN,0
15,Bachelors,2017,A,1,29,Male,No,3,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4634,Bachelors,2016,B,1,38,Female,No,2,1,NaN,NaN,0
4637,Bachelors,2018,A,3,32,Male,Yes,2,1,NaN,NaN,0
4638,Bachelors,2013,A,3,26,Male,No,4,0,NaN,NaN,0
4644,Bachelors,2015,C,3,32,Female,Yes,1,1,NaN,NaN,0


In [25]:
n_correct = (data_test['LeaveOrNot'] == data_test['predict']).sum()
print(n_correct)

accuracy = n_correct/1395
print('accuracy is')
print(accuracy)

904
accuracy is
0.6480286738351254
